In [1]:
# # # Parameters
DT = "2024-06-11"   

## **One Model Profile 생성 작업**

- 개요
    - One Model Profile 생성 작업
- 내용
    - item, cat1, cat2 level로 프로파일링 작업 수행
    - 단기 preference 정보
        - 7일 내 item 정보를 통해서 생성
        - 이때, item은 정규화 로직을 통해 추출된 단어 기반으로 최종 단어들은 추출
    - 장기 preference 정보
        - 60일 내 cat1, cat2 정보를 통해서 생성
        - cat2가 더 상위 레벨임. 어떤 레벨로 장기 구성할지 선택 필요- 내용- 내용

In [2]:
!pip install konlpy

  Using cached konlpy-0.6.0-py2.py3-none-any.whl.metadata (1.9 kB)
  Using cached JPype1-1.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.9 kB)
  Using cached lxml-5.2.2-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.4 kB)
Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
Using cached JPype1-1.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (488 kB)
Using cached lxml-5.2.2-cp310-cp310-manylinux_2_28_x86_64.whl (5.0 MB)


In [3]:
from skt.gcp import get_bigquery_client, bq_insert_overwrite, get_max_part, bq_to_df, bq_to_pandas, pandas_to_bq_table, load_query_result_to_table, df_to_bq_table
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql import functions as F
from datetime import datetime, date, timedelta
from skt.ye import get_spark
import pandas as pd
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [4]:
okt = Okt()

protected_keywords = ['특가']

except_keywords = ['이', '그', '저', '것', '수', '등', '들', '및', '의', '를', '에', '가', '은', '는', '으로', '도', '과', '와', '한', '하고', '에서', '하다',
                   '택', "종", "개", "형", "모음", "세트", "모듬", "모음전", "외", "정"]

def extract_words_by_numbers(words, numbers):
    # 단어와 숫자를 합친 쌍을 만듭니다.
    word_number_pairs = list(zip(words, numbers))

    # 숫자를 기준으로 쌍을 정렬합니다.
    sorted_pairs = sorted(word_number_pairs, key=lambda x: -x[1])

    # 정렬된 쌍에서 단어를 추출합니다.
    sorted_words = [pair[0] for pair in sorted_pairs]

    return sorted_words

def preprocess_text(text):
    # 특수문자와 숫자 제거
    text = re.sub(r'[^가-힣\s]', ' ', text)
    
    # 텍스트를 소문자로 변환
    text = text.lower()
    
    # 단어 정규화
    text = okt.normalize(text)
    
    # 전처리하여 단어 보호
    protect_text = [x for x in text.split() if x in protected_keywords]
    text = [x for x in text.split() if (x not in protected_keywords) & (x not in except_keywords)]
    text = ' '.join(text)
    
    # 명사만 활용
    text = text.split() # okt.nouns(text)
    text.extend(protect_text)
    
    return ' '.join(text)


def extract_keywords_low_idf(texts, num_keywords=10):
    # 텍스트 전처리
    processed_texts = [preprocess_text(text) for text in texts]
    
    # TF-IDF 벡터화
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(processed_texts)
    
    # 단어 순서와 IDF 값을 가져오기
    feature_names = vectorizer.get_feature_names_out()
    idf_values = vectorizer.idf_
    
    keywords = []
    for i in range(len(processed_texts)):
        split_dt = processed_texts[i].split()
        scroe_idf = [-idf_values[feature_names==x][0] if len(idf_values[feature_names==x])>0 else 0 for x in split_dt]
        sorted_indices = np.argsort(scroe_idf)
        sorted_features = [split_dt[idx] for idx in sorted_indices]
        keywords.append(sorted_features[:np.min([num_keywords, len(split_dt)])])
    
    return keywords

# 프로파일 : tdeal_v3

In [5]:
# DT = "2024-06-02"
current_date = datetime.strptime(DT, "%Y-%m-%d")
DT_threshold1 = (current_date - timedelta(days=60)).strftime("%Y-%m-%d")
DT_threshold2 = (current_date - timedelta(days=7)).strftime("%Y-%m-%d")

print("DT : ", DT)
print("DT_threshold : ", DT_threshold1)
print("DT_threshold : ", DT_threshold2)

DT :  2024-06-11
DT_threshold :  2024-04-12
DT_threshold :  2024-06-04


In [6]:
query = f"""
    select *
    from adot_reco.recgpt_log_sequence_lag_daily_prd
    where dt >= '{DT_threshold1}' and type = 'tdeal_v3'
"""

In [7]:
data = bq_to_df(query)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/11 15:50:53 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
24/06/11 15:51:17 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


## tdeal item level

In [9]:
data_item = data.filter(col('dt') >= f'{DT_threshold2}')

In [10]:
data_item = data_item.select(['svc_mgmt_num','item']).na.drop("any", subset='item')

In [12]:
data_item = data_item.groupby('svc_mgmt_num','item').agg(count("*").alias("item_cnt"))
data_item_user = data_item.groupby('svc_mgmt_num').agg(max("item_cnt").alias("item_max_cnt"))
data_item = data_item.join(data_item_user, on='svc_mgmt_num', how="left")
data_item = data_item.withColumn('item_prop', col("item_cnt")/col("item_max_cnt"))
data_item1 = data_item.filter(col('item_prop') > 0.6)

In [ ]:
# data1_item1.show(n=2)

In [ ]:
# data1_item1.count()

In [13]:
data_item2 = data_item1.select('item').distinct()

In [14]:
data_item2_pd = data_item2.toPandas()

In [15]:
item_list = list(data_item2_pd['item'])

- item을 단어 단위로 재가공(정규화 작업)하는 파트

In [ ]:
# keywords_per_text_low_idf = extract_keywords_low_idf(item_list, num_keywords=5)

In [ ]:
# keywords_per_text_low_idf_normalized = extract_keywords_low_idf(item_list, num_keywords=5)

In [16]:
keywords_per_text_low_idf_normalized1 = extract_keywords_low_idf(item_list, num_keywords=5)

- 기존 item에 정규화 작업된 것들 merge

In [17]:
# keywords_idf_nor_pd = pd.DataFrame([item_list,keywords_per_text_low_idf_normalized],index=['item','item_normalized']).T
keywords_idf_nor_pd1 = pd.DataFrame([item_list,keywords_per_text_low_idf_normalized1],index=['item','item_normalized']).T

In [18]:
spark = get_spark()

In [19]:
keywords_idf_nor_spark = spark.createDataFrame(keywords_idf_nor_pd1)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:471: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  arrow_data = [[(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


In [20]:
data_item1 = data_item1.join(keywords_idf_nor_spark, on='item')
data_item1_exploded = data_item1.withColumn("item_normalized", explode("item_normalized"))
item_keyword_counts = data_item1_exploded.groupBy("svc_mgmt_num", "item_normalized").count()
window_item_spec = Window.partitionBy("svc_mgmt_num").orderBy(desc("count"))

top_keywords = item_keyword_counts.withColumn("rank", row_number().over(window_item_spec)) \
                .filter(col("rank") <= 10) \
                .drop("rank")
data_item2 = top_keywords.groupBy("svc_mgmt_num").agg(concat_ws(",",collect_list("item_normalized")).alias("top_item_normalized"))

In [ ]:
# data1_item2.show(n=10)

## tdeal cat level
- cat2(더 상위 카테고리) > cat1

In [21]:
def cat_level_prefer(data, cate="cat1", thre=0.7):
    data = data.select(['svc_mgmt_num',f'{cate}']).na.drop("any", subset=f'{cate}')
    data = data.groupby('svc_mgmt_num',f'{cate}').agg(count("*").alias(f'{cate}_cnt'))
    data_user = data.groupby('svc_mgmt_num').agg(max(f'{cate}_cnt').alias(f'{cate}_max_cnt'))
    data = data.join(data_user, on='svc_mgmt_num', how="left")
    data = data.withColumn(f'{cate}_prop', col(f'{cate}_cnt')/col(f'{cate}_max_cnt'))
    data = data.filter(col(f'{cate}_prop') > thre)
    data = data.groupBy("svc_mgmt_num").agg(concat_ws(", ",collect_list(f'{cate}')).alias(f'{cate}_list'))
    
    return data

- cat1

In [22]:
data_cat1 = cat_level_prefer(data, cate="cat1", thre=0.6)

- cat2

In [23]:
data_cat2 = cat_level_prefer(data, cate="cat2", thre=0.5)

- cat3

In [ ]:
# data1_cat3 = cat_level_prefer(data1, cate="cat3", thre=0.7)

### 전체 데이터 Merge

In [24]:
full_data = data_item2.join(data_cat1,on="svc_mgmt_num")
full_data = full_data.join(data_cat2,on="svc_mgmt_num")
# full_data = full_data.join(data1_cat3,on="svc_mgmt_num")

In [25]:
full_data1 = full_data.withColumn("dt", lit(DT).cast("date"))

In [69]:
full_data1.show(n=10)

+--------------------+---------------------------------+-------------------+--------------------+---------+----------+
|        svc_mgmt_num|              top_item_normalized|          cat1_list|           cat2_list|cat3_list|        dt|
+--------------------+---------------------------------+-------------------+--------------------+---------+----------+
|000097438935ad46c...|               [스파클, 생수, 병]|             [음료]|              [식품]|  [click]|2024-06-04|
|0001a217490e46046...|  [속옷, 캘빈클라인, 목요특가,...|[남성언더웨어/잠옷]|          [패션의류]|  [click]|2024-06-04|
|00066d5a8c7275e3e...|    [조선, 이용, 그랜드, 부산,...|             [숙박]|         [여행/티켓]|  [click]|2024-06-04|
|0007ba4a17fcd1d42...|  [칸디스, 남여공용, 트레이닝,...|         [남성의류]|          [패션의류]|  [click]|2024-06-04|
|0008addabe88ac82d...|   [반팔, 티셔츠, 나이키, 남녀...|         [남성의류]|          [패션의류]|  [click]|2024-06-04|
|000b6fb0e93adfcfa...|   [나이키, 반바지, 미뉴트리, ...|         [남성의류]|[패션잡화, 패션의류]|  [click]|2024-06-04|
|000eabf2c1108ea72...|[라이트스탠다드, 동원에프

In [26]:
full_data1 = full_data1.drop('cat3_list')
full_data1 = full_data1.withColumnRenamed("top_item_normalized", "tdeal_item_prefer")\
                        .withColumnRenamed("cat1_list", "tdeal_cat1_prefer")\
                        .withColumnRenamed("cat2_list", "tdeal_cat2_prefer")

In [27]:
full_data1.printSchema()

root
 |-- svc_mgmt_num: string (nullable = true)
 |-- tdeal_item_prefer: string (nullable = false)
 |-- tdeal_cat1_prefer: string (nullable = false)
 |-- tdeal_cat2_prefer: string (nullable = false)
 |-- dt: date (nullable = true)



### AIDP 저장

In [28]:
dest_dataset = "x1113099"
partitioned_dest_table = "one_model_profile_tdeal"

In [29]:
get_bigquery_client().query(f"""
    CREATE TABLE IF NOT EXISTS {dest_dataset}.{partitioned_dest_table}
    (
        svc_mgmt_num STRING,
        tdeal_item_prefer STRING,
        tdeal_cat1_prefer STRING,
        tdeal_cat2_prefer STRING,
        dt DATE,
    )
    PARTITION BY dt
""").result()

print(f"생성된 테이블 : {dest_dataset}.{partitioned_dest_table}")

생성된 테이블 : x1113099.one_model_profile_tdeal


In [30]:
df_to_bq_table(df=full_data1,
               dataset=dest_dataset,
               table_name=partitioned_dest_table,
               mode="overwrite")